# Artificial Intelligence - COMPSCI4004 2025-2026

## Lab Week 4: Probability and Bayesian Networks <small><small>v20212022a</small></small>

<font color="red">MODEL SOLUTIONS</font>

----

**Aim**

In this lab/tutorial the aim is to: 
* Get hands-on experience with discrete random variables, probabilities and distributions - in Python and on paper!
* Formulate and implement a Bayesian network in Python
* Apply basic inference methods in the network - in Python and on paper!

**Guide**:

The notebook has two main parts and two parts which are extra and can be carried out as self-study:

   - Part I (Q4.0-Q4.3): Basic probability and inference (in Python and on paper). This is a warm-up exercise and should be completed relatively quickly. 
   - Part II (Q4.4): BayesNet I (the key question is Q4.4.4)
   - Part III (Q4.5): BayesNet II [extra; this is open-ended an intended for self-study]  
   - Part IV (Q4.6): A more flexible BayesNet toolbox. 

We also encourage you to work on the quiz during the Lab session.  
   

Each part contains specific tasks - often open-ended questions - that you'll need to carry out to make the notebook run or be able to understand the next steps. These are indicated with:

* <font color=dark-magenta>TASK:</font> This is a task for you to carry out before proceeding. 
* <font color=green>CHECKPOINTS:</font> This indicates a key point you should understand before proceeding. If you're in doubt then ask then consult the lab assistants.
* A basic model solution (marked with <font color=red>SOLUTION</font>) will be provided a week after the Lab session.



---

### Q4.0 Introduction & Housekeeping 

In this exercise we will examine discrete random variables and discrete distributions (i.e., distributions which are easy to enumerate and relate to) and we will implement a Bayesian/belief network in Python.

The exercise relies on the [AIMA toolbox](https://github.com/aimacode/aima-python), Python 3 (download it from Moodle not github!) - and is adapted from the `probability.ipynb` tutorial found in the AIMA toolbox. Note: the relevant files from the AIMA toolbox are provided in the lab 4 zip file.

----

__Prerequisites__

Importing these modules from the provided AIMA module should run without error. If you get an error you probably need to update your Python installation.

* <font color=dark-magenta>TASK:</font> Remember to change the AIMA_TOOLBOX_ROOT variable!

In [41]:
import sys
AIMA_TOOLBOX_ROOT="/Users/debasis/teaching/ai-2023/aima-python-uofg"
sys.path.append(AIMA_TOOLBOX_ROOT)

from utils import (
    product, argmax, element_wise_product, matrix_multiplication,
    vector_to_diagonal, vector_add, scalar_vector_product, inverse_matrix,
    weighted_sample_with_replacement, probability, isclose, normalize
)
import random
from collections import defaultdict

# A specific helper function (no need to understand at this point!)
def extend(s, var, val):
    "Copy the substitution s and extend it by setting var to val; return copy."
    s2 = s.copy()
    s2[var] = val
    return s2

----

### Q4.1 The basics - probability distributions in Python ("warm-up")

In this part, we will examine a basic class for defining probability distributions over discrete random variables.

#### Q4.1.1 Variables and distributions

* <font color=dark-magenta>TASK:</font> What's is a discrete random variable?
* <font color=dark-magenta>TASK:</font> What is a probability distribution?

---

#### Q4.1.2 Variables and distributions in Python
* <font color=dark-magenta>TASK:</font> Skim though the `ProbDist` code below and make sure you understand how the distributions and events are specified. 
___Hint___: We use a class to make it easier to do inference later on but you could also implement a lot of this functionality using e.g.  of this in a basic numpy array


The class `ProbDist` defines a discrete probability distribution. We name our random variable and then assign probabilities to the different values of the random variable. Assigning probabilities to the values works similar to that of using a dictionary with keys being the Value and we assign to it the probability.


In [42]:
class ProbDist:
    """A discrete probability distribution. You name the random variable
    in the constructor, then assign and query probability of values.
    >>> P = ProbDist('Flip'); P['H'], P['T'] = 0.25, 0.75; P['H']
    0.25
    >>> P = ProbDist('X', {'lo': 125, 'med': 375, 'hi': 500})
    >>> P['lo'], P['med'], P['hi']
    (0.125, 0.375, 0.5)
    """

    def __init__(self, varname='?', freqs=None):
        """If freqs is given, it is a dictionary of values - frequency pairs,
        then ProbDist is normalized."""
        self.prob = {}
        self.varname = varname
        self.values = []
        if freqs:
            for (v, p) in freqs.items():
                self[v] = p
            self.normalize()

    def __getitem__(self, val):
        """Given a value, return P(value)."""
        try:
            return self.prob[val]
        except KeyError:
            return 0

    def __setitem__(self, val, p):
        """Set P(val) = p."""
        if val not in self.values:
            self.values.append(val)
        self.prob[val] = p

    def normalize(self):
        """Make sure the probabilities of all values sum to 1.
        Returns the normalized distribution.
        Raises a ZeroDivisionError if the sum of the values is 0."""
        total = sum(self.prob.values())
        if not isclose(total, 1.0):
            for val in self.prob:
                self.prob[val] /= total
        return self

    def show_approx(self, numfmt='{:.3g}'):
        """Show the probabilities rounded and sorted by key, for the
        sake of portable doctests."""
        return ', '.join([('{}: ' + numfmt).format(v, p)
                          for (v, p) in sorted(self.prob.items())])

    def __repr__(self):
        return "P({})".format(self.varname)
    

def event_values(event, variables):
    u"""Return a tuple of the values of variables in event.
    >>> event_values ({'A': 10, 'B': 9, 'C': 8}, ['C', 'A'])
    (8, 10)
    >>> event_values ((1, 2), ['C', 'A'])
    (1, 2)
    """
    if isinstance(event, tuple) and len(event) == len(variables):
        return event
    else:
        return tuple([event[var] for var in variables])

----
#### Q4.1.3 Example: The unfair coin

Consider an unfair coin which has 75% probability of coming out tail. Lets define the probability distribution for Flip, i.e. P(Flip).

* <font color=dark-magenta>TASK:</font> Step through/run the code below and make sure you appreciate how the ProbDist object can be used.


In [43]:
p = ProbDist('Flip')
p['H'], p['T'] = 0.25, 0.75
p['T']

repr(p)

'P(Flip)'

The first parameter of the constructor **varname** has a default value of '?'. So if the name is not passed it defaults to ?. The keyword argument **freqs** can be a dictionary of values of random variable:probability. These are then normalized such that the probability values sum upto 1 using the **normalize** method.

In [44]:
p = ProbDist(freqs={'low': 125, 'medium': 375, 'high': 500})
p.varname

'?'

In [45]:
(p['low'], p['medium'], p['high'])

(0.125, 0.375, 0.5)

Besides the **prob** and **varname** the object also separately keeps track of all the values of the distribution in a list called **values**. Every time a new value is assigned a probability it is appended to this list, This is done inside the **_ _setitem_ _** method.

In [46]:
p.values

['low', 'medium', 'high']

---
#### Q4.1.4 Example - Counting Animals

The distribution by default is not normalized if values are added incremently. We can still force normalization by invoking the **normalize** method. This is relevant when we e.g. count events happing in the world. For example imagine counting the occurances of Mice, Dog and Cat on a farm and then wanting to quantify the probability of seeing each of these animals on the farm.


* <font color=dark-magenta>TASK:</font> Step through/run the code below and make sure you appreciate how the ProbDist object can be used in bith an unnormalized and normalized form.

In [47]:
p = ProbDist('Y')
p['Cat'] = 50
p['Dog'] = 114
p['Mice'] = 64
(p['Cat'], p['Dog'], p['Mice'])

(50, 114, 64)

In [48]:
p.normalize()
(p['Cat'], p['Dog'], p['Mice'])

(0.21929824561403508, 0.5, 0.2807017543859649)

It is also possible to display the approximate values upto decimals using the **show_approx** method.

In [49]:
p.show_approx()

'Cat: 0.219, Dog: 0.5, Mice: 0.281'

----

### Q4.2 Joint Probability Distribution

In this part, we will define a basic Python class for a JOINT probability distributions over __discrete__ random variables, e.g. $P(X,Y)$ where X can be true or false. 

#### Q4.2.1 Basic joint distributions

* <font color=dark-magenta>TASK:</font> Write down (on paper or in this notebook) the two possible variations of the product-rule for the joint distrbution  $P(X,Y)$ ?
* <font color=dark-magenta>TASK:</font> Write down (on paper or in this notebook) the marginalization / sum-rule for $P(X,Y)$ (marginalizing over Y) ? 

<div style="border:2px solid red"></div>
<font color="red">SOLUTION</font>

- $P(X,Y)=P(Y|X)P(X)=P(X|Y)P(Y)$
- $P(X)=\sum_y P(X,Y=y) =\sum_y P(X|Y=y) P(Y=y)$

<div style="border:2px solid red"></div>

----

#### Q4.2.2 Events

The helper function **event_values** returns a tuple of the values of variables in event. An event is specified by a dict where the keys are the names of variables and the corresponding values are the value of the variable. Variables are specified with a list. The ordering of the returned tuple is same as those of the variables.


Alternatively, if the event is specified by a list or tuple of equal length of the variables. Then the events tuple is returned as it is.

In [50]:
event = {'A': 10, 'B': 9, 'C': 8}
variables = ['C', 'A']
event_values(event, variables)

(8, 10)

---

#### Q4.2.3  Joint distribution class

_A probability model is completely determined by the joint distribution for all of the random variables._ (**AIMA Section 13.3**) The function below implements the class **JointProbDist** which inherits from the **ProbDist** class. This class specifies a discrete probability distribute over a set of variables (e.g. X and Y). 

* <font color=dark-magenta>TASK:</font> Skim though the `JointProbDist` code below and make sure you understand how the distributions and events are specified. 
___Hint___: We use a class to make it easier to do inference later on but you could also implement a lot of this functionality using e.g.  of this in a basic numpy array. It is not critical that you undersatnd all aspects of the code but simply accept that it holds the $P(X,Y,...)$-values


In [51]:
class JointProbDist(ProbDist):
    """A discrete probability distribute over a set of variables.
    >>> P = JointProbDist(['X', 'Y']); P[1, 1] = 0.25
    >>> P[1, 1]
    0.25
    >>> P[dict(X=0, Y=1)] = 0.5
    >>> P[dict(X=0, Y=1)]
    0.5"""

    def __init__(self, variables):
        self.prob = {}
        self.variables = variables
        self.vals = defaultdict(list)

    def __getitem__(self, values):
        """Given a tuple or dict of values, return P(values)."""
        values = event_values(values, self.variables)
        return ProbDist.__getitem__(self, values)

    def __setitem__(self, values, p):
        """Set P(values) = p.  Values can be a tuple or a dict; it must
        have a value for each of the variables in the joint. Also keep track
        of the values we have seen so far for each variable."""
        values = event_values(values, self.variables)
        self.prob[values] = p
        for var, val in zip(self.variables, values):
            if val not in self.vals[var]:
                self.vals[var].append(val)

    def values(self, var):
        """Return the set of possible values for a variable."""
        return self.vals[var]

    def __repr__(self):
        return "P({})".format(self.variables)

Values for a Joint Distribution is a an ordered tuple in which each item corresponds to the value associate with a particular variable. For Joint Distribution of X, Y where X, Y take integer values this can be something like (18, 19).

---

#### Q4.2.2 Example

* <font color=dark-magenta>TASK:</font> Step through/run the example code below and make sure you appreciate how the JointProbDist object can be used in defining a joint probability distrbution.

To specify a Joint distribution we first need an ordered list of variables.

In [52]:
variables = ['X', 'Y']
P_XY = JointProbDist(variables)
P_XY

P(['X', 'Y'])

Like the **ProbDist** class **JointProbDist** also employes magic methods to assign probability to different values.
The probability can be assigned in either of the two formats for all possible values of the distribution. The **event_values** call inside  **_ _getitem_ _**  and **_ _setitem_ _** does the required processing to make this work.

In [53]:
P_XY[1,1] = 0.2
P_XY[dict(X=0, Y=1)] = 0.5

(P_XY[1,1], P_XY[0,1])

(0.2, 0.5)

It is also possible to list all the values for a particular variable using the **values** method.

In [54]:
P_XY.values('X')

[1, 0]

----

#### Q4.2.2 Define a joint distrbution for the three variables X,Y,Z

* <font color=dark-magenta>TASK:</font> Define a joint distrbution for the three variables X,Y,Z using the `JointProbDist`

In [15]:
# Insert your code here

<div style="border:2px solid red"></div>
<font color="red">SOLUTION</font>

In [16]:
variables = ['X', 'Y','Z']
P_XYZ = JointProbDist(variables)
P_XYZ

P(['X', 'Y', 'Z'])

In [17]:
P_XYZ[1,1,1] = 0.2
P_XYZ[dict(X=0, Y=1, Z=0)] = 0.5
P_XYZ

(P_XYZ[1,1,0], P_XYZ[1,1,1])

(0, 0.2)

<div style="border:2px solid red"></div>

---

### Q4.3  Inference Using Full Joint Distributions

In this section we use the full Joint Distributions to calculate the posterior distribution given some evidence, i.e. an actual observation of the value of one or more variable (e.g. X=true). 

We represent evidence by using a python dictionary with variables as dict keys and dict values representing the values.

This is illustrated in **AIMA Section 13.3** of the book. The functions **enumerate_joint** and **enumerate_joint_ask** implement this functionality. Under the hood they implement **Equation 13.9** from the AIMA book.

$${P}(X | {e}) = \alpha {P}(X, {e}) = \alpha {P}({e}|X)P(X) = α \sum_{y} {P}(X, {e}, {y})\,\,\,\,\,\, Eq.\,(1)$$

Here $\alpha$ is the normalizing factor. $X$ is our query variable and $e$ is the evidence (i.e. one or more condition variables). According to the equation we enumerate on the remaining variables $y$ (not in evidence or query variable) i.e. all possible combinations of $y$.

---

#### Q4.3.1 Normalizing constant

* <font color=dark-magenta>TASK:</font> Determine an expression for $\alpha$ in Eq (1) (hint use the product rule)

<div style="border:2px solid red"></div>
<font color="red">SOLUTION</font>

From the product rule, i.e., $P(A,B)=P(A|B)P(B)$, we can directly identify $\alpha$ as
$$\alpha = 1 / P(e)$$
which can, in some cases, directly be evaluated as:

$$P(e) = \sum_{x,y} P(X=x,y, {e})$$


<div style="border:2px solid red"></div>

----
#### Q4.3.2 Dentist Example
We will be using the same "dentist" example as the book and lecture, i.e., let us create the full joint distribution from **Figure 13.3**.

* <font color=dark-magenta>TASK:</font> Complete the specification below and validate (manually) that the formulation constitutes a proper joint distribution (how would you do that?).

In [18]:
full_joint = JointProbDist(['Cavity', 'Toothache', 'Catch'])
full_joint[dict(Cavity=True, Toothache=True, Catch=True)]    = 0.108
full_joint[dict(Cavity=True, Toothache=True, Catch=False)]   = 0.012
full_joint[dict(Cavity=True, Toothache=False, Catch=True)]   = 0.072
full_joint[dict(Cavity=True, Toothache=False, Catch=False)]  = 0.008
# Insert your code (four lines missing)


<div style="border:2px solid red"></div>
<font color="red">SOLUTION</font>

In [19]:
full_joint = JointProbDist(['Cavity', 'Toothache', 'Catch'])
full_joint[dict(Cavity=True, Toothache=True, Catch=True)]    = 0.108
full_joint[dict(Cavity=True, Toothache=True, Catch=False)]   = 0.012

full_joint[dict(Cavity=True, Toothache=False, Catch=True)]   = 0.072
full_joint[dict(Cavity=True, Toothache=False, Catch=False)]  = 0.008

full_joint[dict(Cavity=False, Toothache=True, Catch=True)]   = 0.016
full_joint[dict(Cavity=False, Toothache=True, Catch=False)]  = 0.064

full_joint[dict(Cavity=False, Toothache=False, Catch=True)]  = 0.144
full_joint[dict(Cavity=False, Toothache=False, Catch=False)] = 0.576



- Check: It sums to one !

<div style="border:2px solid red"></div>

---

#### Q4.3.3 Computing marginal distrbutions

* <font color=dark-magenta>TASK:</font> Remind yourself of the the marginalisation rule/sum rule for joint distributions and outline how we can we manually find $P(cavity)$ from the joint distribution $P(Cavity, Toothache, Catch)$)

---

#### Q4.3.4 Computing marginal distrbutions in our `JointProb` class 

Let us now look at the **enumerate_joint** function which returns the sum of those entries in $P$ consistent with $e$,provided variables is $P$'s remaining variables (the ones not in $e$). Here, $P$ refers to the full joint distribution. The function uses a recursive call in its implementation. The first parameter **variables** refers to remaining variables. The function in each recursive call keeps on variable constant while varying others.

In [20]:
def enumerate_joint(variables, e, P):
    """Return the sum of those entries in P consistent with e,
    provided variables is P's remaining variables (the ones not in e)."""
    if not variables:
        return P[e]
    Y, rest = variables[0], variables[1:]
    return sum([enumerate_joint(rest, extend(e, Y, y), P)
                for y in P.values(Y)])

Let us assume we want to find **P(Toothache=True)**. This can be obtained by marginalization (**AIMA Equation 13.6**). We can use **enumerate_joint** to solve for this by taking Toothache=True as our evidence. **enumerate_joint** will return the sum of probabilities consistent with evidence i.e. the Marginal Probability (or distribution).

In [21]:
evidence = dict(Toothache=True) 
variables = ['Cavity', 'Catch'] # variables not part of evidence
P_toothache = enumerate_joint(variables, evidence, full_joint)
print("P(Toothache=True)=" +str(P_toothache))

P(Toothache=True)=0.2


Note: we use the lower case notation `P_toothache` to indicate a true outcome of the Toothache random variable. We suggest using `P_nottoothache` to denote Toothache=False.

* <font color=dark-magenta>TASK:</font> Manually verify this results (hint: see the lecture notes).

---

#### Q4.3.5 More complicated marginals
 We can use the same function to find more complex probabilities like **P(Cavity=True and Toothache=True)** 

In [22]:
evidence = dict(Cavity=True, Toothache=True)
variables = ['Catch'] # variables not part of evidence
P_toothachecavity = enumerate_joint(variables, evidence, full_joint)
print("P(Toothache=True,Cavity=True)=" +str(P_toothachecavity))

P(Toothache=True,Cavity=True)=0.12


* <font color=dark-magenta>TASK:</font> Manually verify this result.

---

#### Q4.3.6 Conditional marginals

Being able to sum over probabilities satisfying given evidence allows us to compute conditional probabilities like $P(Cavity=True | Toothache=True)$ as we can rewrite this as 

$$P(Cavity=True\, |\, Toothache = True) = \frac{P(Cavity=True \, , \, Toothache=True)}{P(Toothache=True)}\, \, \, \, Eq. (2)$$

We have already calculated both the numerator and denominator.

In [23]:
P_cavity_toothache = P_toothachecavity/P_toothache
print("P(Cavity=True | Toothache=True )=" +str(P_cavity_toothache))

P(Cavity=True | Toothache=True )=0.6


* <font color=dark-magenta>TASK:</font> Manually verify this result using pen and paper (and perhaps a calculator).

----

#### Q4.3.7 Bayes

* Rewrite Eq (2) into the standard form of Bayes rule, i.e. $P(A|B)=P(B|A)P(A)/(B)$, (insert latex here or do it on paper). Hint: this is almost trivial if you know the product rule.

<div style="border:2px solid red"></div>
<font color="red">SOLUTION</font>

$$P(Cavity=True\, |\, Toothache = True) = \frac{P(Toothache=True \, | \, Cavity=True)  \, P(Cavity=True)}{P(Toothache = True)}$$

<br>

<div style="border:2px solid red"></div>

----

#### Q4.3.8 Implementing Bayes rule in Python via `enumerate_joint_ask`

In the standard form of Bayes rule we are interested in the probability distribution of a particular variable conditioned on some evidence, e.g. $P(Toothache = True\, |\, Cavity=True)$. This can involve doing calculations like above for each possible value of the variable. This has been implemented slightly differently  using normalization in the function **enumerate_joint_ask** which returns a probability distribution over the values of the variable $X$, given the {var:val} observations **e**, in the **JointProbDist P**. The implementation of this function calls **enumerate_joint** for each value of the query variable and passes **extended evidence** with the new evidence having $X=x_i$. This is followed by normalization of the obtained distribution.

In [24]:
def enumerate_joint_ask(X, e, P):
    """Return a probability distribution over the values of the variable X,
    given the {var:val} observations e, in the JointProbDist P. [Section 13.3]
    >>> P = JointProbDist(['X', 'Y'])
    >>> P[0,0] = 0.25; P[0,1] = 0.5; P[1,1] = P[2,1] = 0.125
    >>> enumerate_joint_ask('X', dict(Y=1), P).show_approx()
    '0: 0.667, 1: 0.167, 2: 0.167'
    """
    assert X not in e, u"Query variable must be distinct from evidence"
    Q = ProbDist(X)  # probability distribution for X, initially empty
    Y = [v for v in P.variables if v != X and v not in e]  # hidden variables.
    for xi in P.values(X):
        Q[xi] = enumerate_joint(Y, extend(e, X, xi), P)
    return Q.normalize()

Let us find $P(Cavity | Toothache=True)$ using **enumerate_joint_ask** .

In [25]:
query_variable = 'Cavity'
evidence = dict(Toothache=True)
P_Cavity_Toothache = enumerate_joint_ask(query_variable, evidence, full_joint)
(P_Cavity_Toothache[True], P_Cavity_Toothache[False])

(0.6, 0.39999999999999997)

* <font color=dark-magenta>TASK:</font> You can verify that the first value is the same as we obtained earlier by "manual" calculation.

---

####  Q4.3.9 [optional] sanity check

* <font color=dark-magenta>TASK:</font> For completeness - or a sanity check - compute (using Python) the value for $P(Cavity=True | Toothache=True)$ using the expression for Bayes rule derived in Q4.3.7

In [26]:
# Insert your code here

<div style="border:2px solid red"></div>
<font color="red">SOLUTION</font>

In [27]:
# Get denominator  - P(Toothache)
evidence = dict(Toothache=True)
variables = ['Cavity', 'Catch'] # variables not part of evidence
P_toothache = enumerate_joint(variables, evidence, full_joint)
print("P(Toothache=True )=" +str(P_toothache))

P(Toothache=True )=0.2


In [28]:
# Get numerator I - P(Toothache | Cavity)
query_variable = 'Toothache'
evidence = dict(Cavity=True)
P_Toothache_Cavity = enumerate_joint_ask(query_variable, evidence, full_joint)
(P_Toothache_Cavity[True], P_Toothache_Cavity[False])

(0.6, 0.39999999999999997)

In [29]:
# Get numerator  II - P(Cavity)
evidence = dict(Cavity=True)
variables = ['Toothache', 'Catch'] # variables not part of evidence
P_cavity = enumerate_joint(variables, evidence, full_joint)
print("P(Cavity=True )=" + str(P_cavity))

P(Cavity=True )=0.19999999999999998


In [30]:
# Use the std version of Bayes formula to compute the specific conditional
P_cavity_toothache = (P_Toothache_Cavity[True] * P_cavity)/P_toothache
print("P(Cavity=True | Toothache=True)=" + str(P_cavity_toothache))

P(Cavity=True | Toothache=True)=0.5999999999999999


=> The same as before; as expected!!!

A last sanity check.... check that the joint $P(Toothache=True,Cavity=True)$ is equal to the previosly computed value

In [31]:

P_toothachecavity = (P_Toothache_Cavity[True] * P_cavity)
print("P(Cavity=True , Toothache=True)=" + str(P_toothachecavity))

P(Cavity=True , Toothache=True)=0.11999999999999998


<div style="border:2px solid red"></div>

----

### Q4.4  Bayesian Networks

<font color=red>__Warning:__ This part is more open-ended that Q4.0-Q4.3. While there are suggestions for tasks it up to you to explore the details of the code and applicaitons. </font>

A Bayesian network is a representation of the joint probability distribution encoding a collection of conditional independence statements. This represent our __knowledge__ about these variables based on defined / observed probability distribution.

A Bayes Network is implemented as the class **BayesNet**. It consisits of a collection of nodes implemented by the class **BayesNode**. The implementation in the above mentioned classes focuses only on boolean variables. Each node is associated with a variable and it contains a **conditional probabilty table (cpt)**. The **cpt** represents the probability distribution of the variable conditioned on its parents $P(X_i | parents(X_i))$.

Let us dive into the **BayesNode** implementation (note it is NOT critical that you understand teh details!).

In [32]:

class BayesNode:
    """A conditional probability distribution for a boolean variable,
    P(X | parents). Part of a BayesNet."""

    def __init__(self, X, parents, cpt):
        """X is a variable name, and parents a sequence of variable
        names or a space-separated string.  cpt, the conditional
        probability table, takes one of these forms:

        * A number, the unconditional probability P(X=true). You can
          use this form when there are no parents.

        * A dict {v: p, ...}, the conditional probability distribution
          P(X=true | parent=v) = p. When there's just one parent.

        * A dict {(v1, v2, ...): p, ...}, the distribution P(X=true |
          parent1=v1, parent2=v2, ...) = p. Each key must have as many
          values as there are parents. You can use this form always;
          the first two are just conveniences.

        In all cases the probability of X being false is left implicit,
        since it follows from P(X=true).

        >>> X = BayesNode('X', '', 0.2)
        >>> Y = BayesNode('Y', 'P', {T: 0.2, F: 0.7})
        >>> Z = BayesNode('Z', 'P Q',
        ...    {(T, T): 0.2, (T, F): 0.3, (F, T): 0.5, (F, F): 0.7})
        """
        if isinstance(parents, str):
            parents = parents.split()

        # We store the table always in the third form above.
        if isinstance(cpt, (float, int)):  # no parents, 0-tuple
            cpt = {(): cpt}
        elif isinstance(cpt, dict):
            # one parent, 1-tuple
            if cpt and isinstance(list(cpt.keys())[0], bool):
                cpt = {(v,): p for v, p in cpt.items()}

        assert isinstance(cpt, dict)
        for vs, p in cpt.items():
            assert isinstance(vs, tuple) and len(vs) == len(parents)
            assert all(isinstance(v, bool) for v in vs)
            assert 0 <= p <= 1

        self.variable = X
        self.parents = parents
        self.cpt = cpt
        self.children = []

    def p(self, value, event):
        """Return the conditional probability
        P(X=value | parents=parent_values), where parent_values
        are the values of parents in event. (event must assign each
        parent a value.)
        >>> bn = BayesNode('X', 'Burglary', {T: 0.2, F: 0.625})
        >>> bn.p(False, {'Burglary': False, 'Earthquake': True})
        0.375"""
        assert isinstance(value, bool)
        ptrue = self.cpt[event_values(event, self.parents)]
        return ptrue if value else 1 - ptrue

    def sample(self, event):
        """Sample from the distribution for this variable conditioned
        on event's values for parent_variables. That is, return True/False
        at random according with the conditional probability given the
        parents."""
        return probability(self.p(True, event))

    def __repr__(self):
        return repr((self.variable, ' '.join(self.parents)))

The constructor takes in the name of **variable**, **parents** and **cpt**. Here **variable** is a the name of the variable like 'Earthquake'. **parents** should a list or space separate string with variable names of parents. The conditional probability table is a dict {(v1, v2, ...): p, ...}, the distribution P(X=true | parent1=v1, parent2=v2, ...) = p. Here the keys are combination of boolean values that the parents take. The length and order of the values in keys should be same as the supplied **parent** list/string. In all cases the probability of X being false is left implicit, since it follows from P(X=true).

In the example below we implement the network shown in **AIMA Figure 14.3** of the book :

<img src="./resources/bayesnet.png">

To represent knowlegde in this domain we define a function BayesNet which exploits the BayesNode in defining a full network:

In [33]:
class BayesNet:
    """Bayesian network containing only boolean-variable nodes."""

    def __init__(self, node_specs=[]):
        """Nodes must be ordered with parents before children."""
        self.nodes = []
        self.variables = []
        for node_spec in node_specs:
            self.add(node_spec)

    def add(self, node_spec):
        """Add a node to the net. Its parents must already be in the
        net, and its variable must not."""
        node = BayesNode(*node_spec)
        assert node.variable not in self.variables
        assert all((parent in self.variables) for parent in node.parents)
        self.nodes.append(node)
        self.variables.append(node.variable)
        for parent in node.parents:
            self.variable_node(parent).children.append(node)

    def variable_node(self, var):
        """Return the node for the variable named var.
        >>> burglary.variable_node('Burglary').variable
        'Burglary'"""
        for n in self.nodes:
            if n.variable == var:
                return n
        raise Exception("No such variable: {}".format(var))

    def variable_values(self, var):
        """Return the domain of var."""
        return [True, False]

    def __repr__(self):
        return 'BayesNet({0!r})'.format(self.nodes)

----

#### Q4.4.1 Define a BayeNet for the Burglary example

We will now define the network for your problem:


* <font color=dark-magenta>TASK:</font> Complete the specification below (replace ? with their correct values from Fig 14.2 in the book)


In [34]:
T, F = True, False

burglary = BayesNet([
    ('Burglary', '', 0.001),
    ('Earthquake', '', 0.002),
    ('Alarm', 'Burglary Earthquake',
     {(T, T): 0.95, (T, F): ?, (F, T): 0.29, (F, F): ?}),
    ('JohnCalls', 'Alarm', {T: ?, F: 0.05}),
    ('MaryCalls', 'Alarm', {T: ?, F: ?})
])

SyntaxError: invalid syntax (<ipython-input-34-8b2430293eed>, line 7)

<div style="border:2px solid red"></div>
<font color="red">SOLUTION</font>

<br>

In [35]:
T, F = True, False

burglary = BayesNet([
    ('Burglary', '', 0.001),
    ('Earthquake', '', 0.002),
    ('Alarm', 'Burglary Earthquake',
     {(T, T): 0.95, (T, F): 0.94, (F, T): 0.29, (F, F): 0.001}),
    ('JohnCalls', 'Alarm', {T: 0.90, F: 0.05}),
    ('MaryCalls', 'Alarm', {T: 0.70, F: 0.01})
])


<div style="border:2px solid red"></div>

**BayesNet** method **variable_node** allows to reach **BayesNode** instances inside a Bayes Net. It is possible to extract (and modify the cpt using this method e.g.

In [36]:
burglary.variable_node('Alarm').cpt

{(True, True): 0.95,
 (True, False): 0.94,
 (False, True): 0.29,
 (False, False): 0.001}

----

#### Q4.4.1 Compactness

A Bayes Network is typically a more compact representation of the full joint distribution and like full joint distributions allows us to do inference i.e. answer questions about probability distributions of random variables given some evidence.

* <font color=dark-magenta>TASK:</font> Explain in your own words why a Bayes Network is **typically** a more compact representation that the full joint distrbution ? In what case is it not more compact ?

<div style="border:2px solid red"></div>
<font color="red">SOLUTION</font>

<br>

- Exploiting the conditional independence reduces the space needed to store the various probabilities, in practice typically to a approx. linear scaling in the number of variables.
- In the "worst" case all variables are dependent and there are no savings in representing the joint as a BN compared to a full joint (other that providing a visual representation).

<br>

<div style="border:2px solid red"></div>

----

##### Q4.4.2 Inference by Enumeration (tutorial)

We apply techniques similar to those used for **enumerate_joint_ask** and **enumerate_joint** to draw inference from Bayesian Networks. **enumeration_ask** and **enumerate_all** implement the algorithm described in **Figure 14.9** of the book.

In [37]:
def enumerate_all(variables, e, bn):
    """Return the sum of those entries in P(variables | e{others})
    consistent with e, where P is the joint distribution represented
    by bn, and e{others} means e restricted to bn's other variables
    (the ones other than variables). Parents must precede children in variables."""
    if not variables:
        return 1.0
    Y, rest = variables[0], variables[1:]
    Ynode = bn.variable_node(Y)
    if Y in e:
        return Ynode.p(e[Y], e) * enumerate_all(rest, e, bn)
    else:
        return sum(Ynode.p(y, e) * enumerate_all(rest, extend(e, Y, y), bn)
                   for y in bn.variable_values(Y))

**enumerate__all** recursively evaluates a general form of the **Equation 14.4** in the book.

$${P}(X | {e}) = α {P}(X, {e}) = α \sum_{y} {P}(X, {e}, {y})$$ 

such that $P(X, e, y)$ is written in the form of product of conditional probabilities **P(variable | parents(variable))** from the Bayesian Network.

**enumeration_ask** calls **enumerate_all** on each value of query variable $X$ and finally normalizes them. 


In [38]:
def enumeration_ask(X, e, bn):
    u"""Return the conditional probability distribution of variable X
    given evidence e, from BayesNet bn. [Figure 14.9]
    >>> enumeration_ask('Burglary', dict(JohnCalls=T, MaryCalls=T), burglary
    ...  ).show_approx()
    'False: 0.716, True: 0.284'"""
    assert X not in e, u"Query variable must be distinct from evidence"
    Q = ProbDist(X)
    for xi in bn.variable_values(X):
        Q[xi] = enumerate_all(bn.variables, extend(e, X, xi), bn)
    return Q.normalize()

Let us solve the problem of finding out $P(Burglary=True | JohnCalls=True, MaryCalls=True)$ using the **burglary** network. **enumeration_ask** takes three arguments $X$ = variable name, $e$ = Evidence (in form a dict like previously explained), **bn** = The Bayes Net to do inference on.

In [39]:
P_B_JM = enumeration_ask('Burglary', {'JohnCalls': True, 'MaryCalls': True}, burglary)
(P_B_JM[True],P_B_JM[False])

(0.2841718353643929, 0.7158281646356071)

---

##### Q4.4.4 Manual inference in the Bayesian network
* <font color=dark-magenta>TASK:</font> Manually (without Python) compute the distribution for $P(Burglary\,,\,MaryCalls\,|\,JohnCalls)$ ) (you don't need to insert the numerical expressions; only the relevant conditional distributions). Check with your lab assistant if you have got it right!

Hint: you will often be asked to do something similar in the exam.

<div style="border:2px solid red"></div>
<font color="red">SOLUTION</font>
<br>

We provide a general recipe for computing any conditional from any Bayes Net. <br>

**Step 1)** Always write up the conditional exploiting all conditional independencies: <br>

In this case: 

$$P(B,E,A,M,J) = P(E)P(B)P(A|E,B)P(M|A)P(J|A)$$


**Step 2)** Figure out what expression you're looking for.<br>
In this case we are looking for $P(B,M|J)$. 

Step 2.a) Which variables are involved and write the joint for that?<br>
Here P(B,M,J).

Step 2.b) Figure out how you can get to the expression (typ. a conditional) from the joint which can be done with Bayes rule or from the production rules.

Here P(B,M,J)=P(B,M|J)P(J) which leads to 

$$P(B,M|J) = \frac{P(B,M,J)}{P(J)}$$

Note in some case it can be benificial to write $P(B,M|J)=\frac{P(J|B,M)P(B,M)}{P()J}$ (but not nessesary).


**Step 3)** Find the specific expression required to compute the expression found in Step 2 by marginalising out variables from the joint identified in Step 1.
Here we need to find P(B,M,J) and P(J). 

$$\displaylines{
  P(B,M,J) &=& \sum\limits_{a \in \{ T,F\} }^{} {\sum\limits_{e \in \{ T,F\} }^{} {P(B,E = e,A = a,M,J)} }  \cr 
   &=& \sum\limits_{a \in \{ T,F\} }^{} {\sum\limits_{e \in \{ T,F\} }^{} {\left( {P(E = e)P(B)P(A = a|E = e,B)P(M|A = a)P(J|A = a)} \right)} }  \cr 
   &=& P(B)\left( {\sum\limits_{a \in \{ T,F\} }^{} {P(M|A = a)P(J|A = a)\left( {\sum\limits_{e \in \{ T,F\} }^{} {P(E = e)P(A = a|E = e,B)} } \right)} } \right) \cr} $$

<br>
... and P(J) can be found by (or you can start from the full joint):

$$\displaylines{
  P(J) &=& \sum\limits_{m \in \{ T,F\} }^{} {\sum\limits_{b \in \{ T,F\} }^{} {P(B = b,M = m,J)} }  \cr 
   &=& ... \cr} $$

And we can insert these in the expresison found in the Step 2 (sometimes this leads to further opputunity for reductions , but not in general).

<br>

<br>

Sometimes we are lucky and can easily identify the final result without using this mechnical approach; however, the outlined procedure gives you a fail safe approach (see also Ch 14 in the book).

<br>

<div style="border:2px solid red"></div>

---

##### Q4.4.4 Inference in the Bayesian network (using Python)

In order to demonstrate very interesting properties of Bayesian Networks related to conditional independence and "explaining away", we will now compute a number of marginal and conditional distributions to highlight these properties. We are interested in investigating which variables influence each other and under which conditions!

<img src="./resources/bayesnet.png">

We will be asking the following four questions:
* Assume we have ***not*** observed whether the Alarm has gone off or not, will knowing $MaryCalls=true$ influence the probability of $JohnCalls$? (hint: consider the definition of conditional independence; variables are only independent when we have observed something about a parent!).)
* Assume we have observed `Alarm=true`, will knowing $MaryCalls=true$ influence the probability of $JohnCalls$? (hint: consider the definition of conditional independence; variables are only independent when we have observed something about a parent!).
* Assume we have ***not*** observed whether the Alarm has gone off or not, will knowing $Burglary=true$ influence the probability of $Earthquake$? (hint: investigate - using the book, internet etc- the concept of "explaining away").
* Assume we have observed `Alarm=true`, will knowing $Burglary=true$ influence the probability of $JohnCalls$? (hint: investigate - using the book, internet etc- the concept of "explaining away").


* <font color=dark-magenta>TASK:</font> Try to answer the above questions using your intuition before doing the numerical computations.


* <font color=dark-magenta>TASK:</font> To investigate the concept of "explaining away", we suggest you compute the following distributions to explore the above questions:
    * Compute $P(JohnCalls)$
    * Compute $P(MaryCalls)$
    * Compute $P(JohnCalls\,|\,MaryCalls=true)$
    * Compute $P(MaryCalls\,|\,JohnCalls=true)$
    * Compute $P(JohnCalls\,|\,Alarm=true)$
    * Compute $P(MaryCalls\,|\,Alarm=true)$    
    * Compute $P(JohnCalls\,|\,Alarm=true,MaryCalls=true)$
    * Compute $P(MaryCalls\,|\,Alarm=true,JohnCalls=true)$


* <font color=dark-magenta>TASK:</font> To investigate the concept of conditional independence, we suggest you compute the following distributions to explore the above questions:   
    * Compute $P(Burglary)$
    * Compute $P(Earthquake)$      
    * Compute $P(Burglary\,|\,Earthquake=true)$
    * Compute $P(Earthquake\,|\,Burglary=true)$      
    * Compute $P(Burglary\,|\,Alarm=true)$
    * Compute $P(Earthquake\,|\,Alarm=true)$      
    * Compute $P(Burglary\,|\,Alarm=true,Earthquake=true)$
    * Compute $P(Earthquake\,|\,Alarm=true,Burglary=true)$ [hint: this value should be around 0.0033]   
    
    
* <font color=dark-magenta>TASK:</font> Return to the four questions raised above and check that the numerical values  values aligns with your understanding. 
    
    
* <font color=green>CHECKPOINT</font>: Make sure you (as a group) discuss these aspects internally and with the tutor (or at least read through the solution when available!)
      

<div style="border:2px solid red"></div>
<font color="red">SOLUTION</font>
<br>





In [40]:
print("We use a short-hand notation for the variable names, s.t. P_X_yz = P(X|Y=true,Z=true)")

print("--------------------------------------------------")
print("BOTTOM: Diverging connection, cond. independence")
print("--------------------------------------------------")
print("  \t  < True , False >")

P_J = enumeration_ask('JohnCalls', {}, burglary)
print("P_J\t= <%.4f, %.4f>" % (P_J[True],P_J[False]))

P_M = enumeration_ask('MaryCalls', {}, burglary)
print("P_M\t= <%.4f, %.4f>" % (P_M[True],P_M[False]))

P_J_m = enumeration_ask('JohnCalls', {'MaryCalls': True}, burglary)
print("P_J_m\t= <%.4f, %.4f>" % ( P_J_m[True],P_J_m[False]))

P_M_j = enumeration_ask('MaryCalls', {'JohnCalls': True}, burglary)
print("P_M_j\t= <%.4f, %.4f>" % ( P_M_j[True],P_M_j[False]))

# with alarm
P_J_a = enumeration_ask('JohnCalls', {'Alarm': True}, burglary)
print("P_J_a\t= <%.4f, %.4f>" % ( P_J_a[True],P_J_a[False]))

P_M_a = enumeration_ask('MaryCalls', {'Alarm': True}, burglary)
print("P_M_a\t= <%.4f, %.4f>" % ( P_M_a[True],P_M_a[False]))

# with alarmn and other caller
P_J_ma = enumeration_ask('JohnCalls', {'Alarm': True,'MaryCalls': True}, burglary)
print("P_J_ma\t= <%.4f, %.4f>" % ( P_J_ma[True],P_J_ma[False]))

P_M_ja = enumeration_ask('MaryCalls', {'Alarm': True,'JohnCalls': True}, burglary)
print("P_M_ja\t= <%.4f, %.4f>" % ( P_M_ja[True],P_M_ja[False]))

print("\n\n")

print("--------------------------------------------------")
print("TOP: converging connections, explaining away")
print("--------------------------------------------------")
print(" \t  < True , False >")
P_E = enumeration_ask('Earthquake', {}, burglary)
print("P_E\t= <%.4f, %.4f>" % ( P_E[True],P_E[False]))

P_B= enumeration_ask('Burglary', {}, burglary)
print("P_B\t= <%.4f, %.4f>" % ( P_B[True],P_B[False]))

P_B_e = enumeration_ask('Burglary', {'Earthquake': True}, burglary)
print("P_B_e\t= <%.4f, %.4f>" % (P_B_e[True], P_B_e[False]))

P_E_b = enumeration_ask('Earthquake', {'Burglary': True}, burglary)
print("P_E_b\t= <%.4f, %.4f>" % (P_E_b[True], P_E_b[False]))

P_B_a = enumeration_ask('Burglary', {'Alarm': True}, burglary)
print("P_B_a\t= <%.4f, %.4f>" % ( P_B_a[True],P_B_a[False]))

P_E_a = enumeration_ask('Earthquake', {'Alarm': True}, burglary)
print("P_E_a\t= <%.4f, %.4f>" % ( P_E_a[True],P_E_a[False]))


P_E_ba= enumeration_ask('Earthquake', {'Alarm': True, 'Burglary': True}, burglary)
print("P_E_ba\t= <%.4f, %.4f>" % (P_E_ba[True], P_E_ba[False]))

P_B_ea= enumeration_ask('Burglary', {'Alarm': True, 'Earthquake': True}, burglary)
print("P_B_ea\t= <%.4f, %.4f>" % (P_B_ea[True], P_B_ea[False]))





We use a short-hand notation for the variable names, s.t. P_X_yz = P(X|Y=true,Z=true)
--------------------------------------------------
BOTTOM: Diverging connection, cond. independence
--------------------------------------------------
  	  < True , False >
P_J	= <0.0521, 0.9479>
P_M	= <0.0117, 0.9883>
P_J_m	= <0.1776, 0.8224>
P_M_j	= <0.0400, 0.9600>
P_J_a	= <0.9000, 0.1000>
P_M_a	= <0.7000, 0.3000>
P_J_ma	= <0.9000, 0.1000>
P_M_ja	= <0.7000, 0.3000>



--------------------------------------------------
TOP: converging connections, explaining away
--------------------------------------------------
 	  < True , False >
P_E	= <0.0020, 0.9980>
P_B	= <0.0010, 0.9990>
P_B_e	= <0.0010, 0.9990>
P_E_b	= <0.0020, 0.9980>
P_B_a	= <0.3736, 0.6264>
P_E_a	= <0.2310, 0.7690>
P_E_ba	= <0.0020, 0.9980>
P_B_ea	= <0.0033, 0.9967>


* Conditional independence (the bottom diverging connection): When we do not know the specific value of Alarm the two values influence each other (i.e. both goes up compared to the base values i.e. $P(j) < P(j|m)$. When we know Alarm=true the two values naturally revert to the conditional probabilities. 


* "Explaining away" (the top converging connection): There are two reasons for the Alarm going off ($E$ and $B$). Without knowing if Alarm=true the two variables do not influence each other. When we know A=true both $P(E|A)$ and $P(B|A)$ goes up. However, when we observe one of the causal reason to be true the e.g. $B=true$ the other reason (here $P(E|B=true,A=true)$ ) reverts back to a low probability as we have already "explained" why the Alarm went off (albeit it doesn't revert to independent conditional, $P(E)$,  as the Alarm=true and there is still uncertainty as to the whether the there was also a Earthquake. They now influence each other through the Alarm node. The results is intuitive if you think about it...

    

<div style="border:2px solid red"></div>

----

### Q4.5 BayesNet II [extra; self-study]

You are asked to determine if adaptive (”cheating”) software is active in a specific car to circumvent environmental requirements. The software is suspected to change the operation of the engine conditioned on the environment (i.e., control or real environment). The knowledge for a particular manufacturer and model is shown in the BayesNet below and the variables
are outlined in the following table:

<img src="./resources/car2vars.png" width="50%">

<img src="./resources/fn_bn_cars_r002b.png" width="70%">

<br>


* <font color=dark-magenta>TASK:</font>Manually determine an expression for the joint distribution $P(N,C,S,D,E)$ exploiting all the conditional independencies implied by the Bayesian network in the figure.


* <font color=dark-magenta>TASK:</font>Compute the how much memory we save by storing the conditionals compared to the full joint distribution?


* <font color=dark-magenta>TASK:</font>Manually determine an expression an numerical value for the probability of the software not being active given that we have observed the fuel consumption to be below the limit and the NOx level to be above the limit. The expression should allow numerical evaluation given the values.


<div style="border:2px solid red"></div>
<font color="red">SOLUTION</font>
<br>
See Quiz Week 4
<div style="border:2px solid red"></div>



* <font color=dark-magenta>TASK:</font>Implement a BayesNet using the AIMA toolbox to automatically determine the numerical value.

Forward you solution to the lecturer for validation (no solution will be provided upfront).
        

----

### Q4.6 A more flexible library [extra;self-study]
* <font color=dark-magenta>TASK:</font> The AIMA BayesNet toolbox is slightly limited in its functionality. It you are interested in building more advanced Bayesian Networks we recommend you try to recreate a simple BayesNet (e.g. the Earthquake example) in the `pomegranate` library (https://pomegranate.readthedocs.io/en/latest/install.html). Discuss your finding and experience with the tutors/lab assistants. What can you do in `pomegranate`that you can't do with the AIMA toolbox (hint: consider the type of variables to can model in AIMA and compare with `pomegranate`?